# Project 1 – Charles – Metrics

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib
import seaborn as sns

# Adding Session ID

In [2]:
# do not run this entire section if your dataset already has unique ID

In [3]:
#data = pd.read_csv("Cleaned_Data.csv", low_memory=False)
#df = data.sort_values(by=['distinct_id', 'time'], axis=0, ascending=True)
#df['ae_session_length'] = df['ae_session_length'].fillna('-1')
#df['ae_session_length'] = df['ae_session_length'].str.replace(',', '')
#df['ae_session_length'] = df['ae_session_length'].astype(float)
#df['time'] = df['time'] = pd.to_datetime(df['time'])
#df

In [4]:
def add_session_id(df):
    df['session_id'] = -1
    sessions = df[['name_x', 'distinct_id', 'time', 'ae_session_length']].loc[df['name_x'] == '$ae_session']
    sessions['start'] = sessions['time'] - np.array([pd.Timedelta(seconds=np.ceil(x)) for x in sessions['ae_session_length']])
    curr_id = 1
    for index, row in sessions.iterrows():
        df.loc[(df['session_id'] == -1) 
               & (df['time'] <= row['time']) 
               & (df['time'] >= row['start']) 
               & (df['distinct_id'] == row['distinct_id']), 'session_id'] = curr_id
        curr_id += 1

#add_session_id(df)
#df
    

In [5]:
#df.loc[df['session_id'] == -1]

In [6]:
#df.to_csv("Cleaned_Dataset_With_ID.csv")

# Adding Time Between Sessions

In [117]:
data = pd.read_csv("Cleaned_Dataset_With_ID.csv", low_memory=False)

In [118]:
data.drop(columns=['Unnamed: 0'], axis=1, inplace=True)


In [119]:
data = data.sort_values(by=['distinct_id', 'time'], axis=0, ascending=True)
data['time'] = pd.to_datetime(data['time'])
data

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
0,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0.000000e+00
1,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0.000000e+00
2,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0.000000e+00
3,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0.000000e+00
4,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190366,Feed - Scroll Down,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:23,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,3.000000e-09
190367,Profile - Change Section,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:28,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,3.000000e-09
190368,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:30:54,190.4,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,3.000000e-09
190369,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29814,0.000000e+00


In [120]:
grouped = data.loc[data['session_id'] != -1].groupby(['distinct_id', 'session_id'], as_index=False).agg({'time' : ['min', 'max']})
grouped.head()

distinct_id session_id                time  \
                                                                   min   
0  000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          1 2019-08-23 13:48:32   
1  000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          2 2019-08-23 13:50:06   
2  000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          3 2019-08-23 13:50:58   
3  000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          4 2019-08-23 14:37:20   
4  000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD          5 2019-08-23 15:02:56   

                       
                  max  
0 2019-08-23 13:49:02  
1 2019-08-23 13:50:51  
2 2019-08-23 13:51:52  
3 2019-08-23 14:38:30  
4 2019-08-23 15:05:10

In [121]:
grouped.iloc[1, 2] - grouped.iloc[0, 3]

Timedelta('0 days 00:01:04')

In [122]:
grouped.iloc[0,3] - grouped.iloc[0,2]

Timedelta('0 days 00:00:30')

In [123]:
def add_time_delta(df):
    df['time_delta'] = 0
    grouped = df.loc[df['session_id'] != -1].groupby(['distinct_id', 'session_id'], as_index=False).agg({'time' : ['min', 'max']})
    curr_distinct_id = ""
    for index, row in grouped.iterrows():
        if row[0] != curr_distinct_id:
            curr_distinct_id = row[0]
        else:
            delta = row[2] - grouped.iloc[index - 1, 3]
            df.loc[(df['distinct_id'] == row[0]) 
                   & (df['session_id'] == row[1]), 'time_delta'] = delta

add_time_delta(data)
data

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
0,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
1,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
2,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
3,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
4,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190366,Feed - Scroll Down,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:23,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,0 days 00:00:03
190367,Profile - Change Section,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:28,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,0 days 00:00:03
190368,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:30:54,190.4,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29813,0 days 00:00:03
190369,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29814,0


In [138]:
max(data['time_delta'])

18289482.0

In [139]:
def recover(function):
    import inspect
    get_ipython().set_next_input("".join(inspect.getsourcelines(function)[0]))


In [140]:
data.loc[data['time_delta'] == 0]

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
0,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
1,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,-1,-1,NaN,...,NaN,NaN,INTL,0,0,0,0,0,-1,0
2,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
3,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
4,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190349,Sidebar - Go to Feedback,xbarbarazhong@gmail.com_6704,2019-12-12 14:27:39,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29812,0
190350,Visit App Feedback Page,xbarbarazhong@gmail.com_6704,2019-12-12 14:27:39,-1.0,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29812,0
190351,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:27:43,57.9,Portland,Oregon,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29812,0
190369,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,-1,-1,NaN,...,NaN,NaN,US,1,0,0,0,0,29814,0


In [141]:
data.to_csv("Cleaned_Dataset_With_ID.csv")

# Adding Slope

In [142]:
data = pd.read_csv("Cleaned_Dataset_With_ID.csv", low_memory=False)
df = data.loc[(data['name_x'] == '$ae_session') & (data['time_delta'] != 0)]
df

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
19,19,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:51,65.8,Palo Alto,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,2,64.0
21,21,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:51:52,59.9,Palo Alto,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,3,7.0
24,24,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 14:38:30,72.3,Hayward,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,4,2728.0
36,36,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:05:10,151.9,San Jose,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,5,1466.0
50,50,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:08:11,115.8,San Jose,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,6,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190340,190340,$ae_session,willwojt@gmail.com_6868,2020-03-07 12:11:54,56.5,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,29807,40862.0
190341,190341,$ae_session,willwojt@gmail.com_6868,2020-03-07 12:14:44,32.9,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,29808,170.0
190342,190342,$ae_session,willwojt@gmail.com_6868,2020-03-08 10:47:21,42.3,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,29809,81157.0
190343,190343,$ae_session,willwojt@gmail.com_6868,2020-03-08 10:52:05,200.6,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,29810,284.0


In [143]:
max(df['time_delta'])

18289482.0

In [144]:
def best_fit_slope(xs,ys):
    m = (((np.mean(xs)*np.mean(ys)) - np.mean(xs*ys)) /
         ((np.mean(xs)*np.mean(xs)) - np.mean(xs*xs)))
    return m

In [145]:
grouped = df.groupby('distinct_id', as_index=False).agg({'time' : 'count'})
grouped

,distinct_id,time
0,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,10
1,001210DC-54C8-43AD-B295-148F47818391,9
2,0025A8A4-2590-4ECA-8CE8-419D710AE46F,7
3,004C9C76-A530-422A-BA1B-667CDA12A08D,3
4,005144D7-BD3F-405B-9621-5FACA6B1AC04,1
...,...,...
2617,varun.murthy@gmail.com_5928,5
2618,virtrutest99@gmail.com_5848,8
2619,wandaandreu@gmail.com_5053,2
2620,willwojt@gmail.com_6868,98


In [147]:
dic = {}
for index, row in grouped.iterrows():
    x = df.loc[df['distinct_id'] == row['distinct_id']]
    length = np.arange(len(x))
    slope = best_fit_slope(length, x['time_delta'])
    dic[row['distinct_id']] = slope

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [148]:
dic

{'000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD': 1113.3636363636367,
 '001210DC-54C8-43AD-B295-148F47818391': -11716.0,
 '0025A8A4-2590-4ECA-8CE8-419D710AE46F': 615549.607142857,
 '004C9C76-A530-422A-BA1B-667CDA12A08D': -44.49999999998544,
 '005144D7-BD3F-405B-9621-5FACA6B1AC04': nan,
 '00708931-5C43-412D-9ECB-6A8839F38179': 7.0,
 '00782604-A3A7-4434-8167-3C05AD24637D': 1301255.0,
 '00861508-53FF-4D0A-BB62-D9B35381886C': nan,
 '00CE2C99-8AC3-433E-8611-944CAC725EA5': 4015617.0,
 '00D88292-02E5-4966-8079-DB96622C3970': 21540.090909090795,
 '00E3AA4B-FCAC-4197-A767-57802EF9C03C': 1401923.3,
 '00F975B5-34ED-4BE1-AC5B-C7577882EA56': -2177253.0,
 '00d4b8b407679e94': 2.5000000000000067,
 '0111f0fd-4aa6-4031-82a4-fd72a583e9fd': nan,
 '01195431-5FE0-46CB-8EE9-DFF4EBFA54E7': -9680079.0,
 '0127D3CF-D6AB-4802-9255-957D80611839': 2528.0,
 '01301A7F-CD6D-47AA-B975-503694662ED2': 274010.8214285715,
 '015CA5D1-7E4C-413B-9D9B-16A23D0FC0F1': -648.0,
 '015E691D-9082-4A1C-BE29-416933E0DA26': -11450925.0,
 '01838F

In [64]:
df.head(40)

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,...,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta,seconds
19,19,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:51,65.8,Palo Alto,California,US,-1,-1,...,NaN,US,1,0,0,0,0,2,0 days 00:00:45,45.0
21,21,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:51:52,59.9,Palo Alto,California,US,-1,-1,...,NaN,US,1,0,0,0,0,3,0 days 00:00:54,54.0
24,24,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 14:38:30,72.3,Hayward,California,US,-1,-1,...,NaN,US,1,0,0,0,0,4,0 days 00:01:10,70.0
36,36,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:05:10,151.9,San Jose,California,US,-1,-1,...,NaN,US,1,0,0,0,0,5,0 days 00:02:14,134.0
50,50,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:08:11,115.8,San Jose,California,US,-1,-1,...,NaN,US,1,0,0,0,0,6,0 days 00:01:46,106.0
53,53,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:08:59,47.3,San Jose,California,US,-1,-1,...,NaN,US,1,0,0,0,0,7,0 days 00:00:42,42.0
60,60,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:50:32,42.9,Los Altos,California,US,-1,-1,...,NaN,US,1,0,0,0,0,8,0 days 00:00:43,43.0
69,69,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:51:21,37.9,Los Altos,California,US,-1,-1,...,NaN,US,1,0,0,0,0,9,0 days 00:00:36,36.0
78,78,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:53:39,101.1,Los Altos,California,US,-1,-1,...,NaN,US,1,0,0,0,0,10,0 days 00:01:29,89.0
98,98,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:57:06,186.4,Los Altos,California,US,-1,-1,...,NaN,US,1,0,0,0,0,11,0 days 00:02:56,176.0
